In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "wobbly-star-128"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        high_contradiction = get_idx_with_top_column_values('contradiction_score',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","pp_letter_diff", "contradiction_score", "reward","label_flip", 'pp_logp',
       'ref_logp', 'kl_div', 'reward_with_kl', 'loss']
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'training_step'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=2)
print_interesting_text_stats(df_d[split], n=2)


###############

random 0 

Original: I love this apple
Original label 1
Unique paraphrases: 255
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,pp_logp,ref_logp,kl_div,reward_with_kl,loss,
I love that apple,0.80974,0.18908,0.96540,0,0.000453,2.26899,0,-1.188778,-4.461471,3.272693,2.236259,2.658415,[1]
I love this person.,0.92272,0.07610,0.35723,-2,0.887264,0.00000,0,-1.121463,-2.949469,1.828005,0.000000,0.000000,[2]
This apple is very pleasurable.,0.93026,0.06856,0.88949,-14,0.000298,0.82276,0,-0.986968,-4.826566,3.839598,0.784368,0.774146,[3]
.....................,0.44856,0.55026,0.17395,-4,0.023024,0.00000,1,-1.002480,-5.290613,4.288134,0.000000,0.000000,[4]
This apple very dear to my heart.,0.89870,0.10012,0.89389,-16,0.000182,1.20141,0,-1.488593,-5.486300,3.997707,1.161431,1.728898,[5]
The apple I love the most.,0.90529,0.09354,0.89775,-9,0.000365,1.12243,0,-1.003109,-4.502164,3.499055,1.087437,1.090817,[6]
"My friends, I love this apple.",0.91759,0.08123,0.97400,-13,0.000364,0.97473,0,-1.300427,-4.344928,3.044501,0.944285,1.227974,[7]
I have a bunch of apples.,0.25899,0.73983,0.55411,-8,0.001458,0.00000,1,-1.351256,-4.147201,2.795945,0.000000,0.000000,[8]
I like this apple.,0.80203,0.19679,0.95745,-1,0.000089,2.36152,0,-0.424762,-4.499224,4.074462,2.320772,0.985776,[9]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
897,2,398,I love this apple,I adore the apple,0.998821,0.71863,0.71863,1,1,0,0.28019,0.85921,0,1,0.000102738,3,-1.11254,-6.73547,5.62293,2.94377,3.27507,0,397,1,6.06228,1.51557,0,8,2,8,2,0.136439,0.278838,0.0491894,0.00677963,0.0270955,9.25679e-05,0.0146129,0.00629844,0.00129995,0.00354538,0.222973,0.000224662,0.187217,0.0367329,0.170169,0.104155,255,484,4,4,1,0,0,14,4,1,0,0,14,0,0,0,0,0,"[1, 2]",[love this],"[3, 4]",[adore the],"[0, 5]","[I, apple]",1,1,2,2,False,False,False,2



###############

random 1 

Original: I like this movie
Original label 1
Unique paraphrases: 235
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
897,2,398,I love this apple,I adore the apple,0.998821,0.71863,0.71863,1,1,0,0.28019,0.85921,0,1,0.000102738,3,-1.11254,-6.73547,5.62293,2.94377,3.27507,0,397,1,6.06228,1.51557,0,8,2,8,2,0.136439,0.278838,0.0491894,0.00677963,0.0270955,9.25679e-05,0.0146129,0.00629844,0.00129995,0.00354538,0.222973,0.000224662,0.187217,0.0367329,0.170169,0.104155,255,484,4,4,1,0,0,14,4,1,0,0,14,0,0,0,0,0,"[1, 2]",[love this],"[3, 4]",[adore the],"[0, 5]","[I, apple]",1,1,2,2,False,False,False,2



###############

high_contradiction 1 

Original: I love this apple
Original label 1
Unique paraphrases: 255
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,pp_logp,ref_logp,kl_div,reward_with_kl,loss,
I love that apple,0.80974,0.18908,0.96540,0,0.000453,2.26899,0,-1.188778,-4.461471,3.272693,2.236259,2.658415,[1]
I love this person.,0.92272,0.07610,0.35723,-2,0.887264,0.00000,0,-1.121463,-2.949469,1.828005,0.000000,0.000000,[2]
This apple is very pleasurable.,0.93026,0.06856,0.88949,-14,0.000298,0.82276,0,-0.986968,-4.826566,3.839598,0.784368,0.774146,[3]
.....................,0.44856,0.55026,0.17395,-4,0.023024,0.00000,1,-1.002480,-5.290613,4.288134,0.000000,0.000000,[4]
This apple very dear to my heart.,0.89870,0.10012,0.89389,-16,0.000182,1.20141,0,-1.488593,-5.486300,3.997707,1.161431,1.728898,[5]
The apple I love the most.,0.90529,0.09354,0.89775,-9,0.000365,1.12243,0,-1.003109,-4.502164,3.499055,1.087437,1.090817,[6]
"My friends, I love this apple.",0.91759,0.08123,0.97400,-13,0.000364,0.97473,0,-1.300427,-4.344928,3.044501,0.944285,1.227974,[7]
I have a bunch of apples.,0.25899,0.73983,0.55411,-8,0.001458,0.00000,1,-1.351256,-4.147201,2.795945,0.000000,0.000000,[8]
I like this apple.,0.80203,0.19679,0.95745,-1,0.000089,2.36152,0,-0.424762,-4.499224,4.074462,2.320772,0.985776,[9]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,ref_logp,kl_div,reward_with_kl,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
897,2,398,I love this apple,I adore the apple,0.998821,0.71863,0.71863,1,1,0,0.28019,0.85921,0,1,0.000102738,3,-1.11254,-6.73547,5.62293,2.94377,3.27507,0,397,1,6.06228,1.51557,0,8,2,8,2,0.136439,0.278838,0.0491894,0.00677963,0.0270955,9.25679e-05,0.0146129,0.00629844,0.00129995,0.00354538,0.222973,0.000224662,0.187217,0.0367329,0.170169,0.104155,255,484,4,4,1,0,0,14,4,1,0,0,14,0,0,0,0,0,"[1, 2]",[love this],"[3, 4]",[adore the],"[0, 5]","[I, apple]",1,1,2,2,False,False,False,2


In [ ]:
df_d['training_step']

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,...,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
0,0,1,I like this movie,this was a good movie.,0.999343,0.92334,0.923342,1,1,0,...,"[2, 6]","[this, movie]",2,1,4,2,False,False,False,5
1,0,2,I like this movie,I like the movie.,0.999343,0.77841,0.778408,1,1,0,...,"[0, 1, 4]","[I like, movie]",2,1,2,1,False,False,False,2
2,0,3,I like this movie,Themovie is in my opinion a good movie,0.999343,0.78882,0.788824,1,1,0,...,[10],[movie],1,1,7,3,False,False,False,7
3,0,4,I like this movie,There is a film I like about.,0.999343,0.74781,0.747812,1,1,0,...,"[4, 5]",[I like],2,1,6,2,True,False,False,6
4,0,5,I like this movie,I enjoyed this movie.,0.999343,0.91921,0.919212,1,1,0,...,"[0, 3, 4]","[I, this movie]",2,1,2,1,False,False,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,496,I love this apple,I enjoy apples.,0.998821,0.86797,0.867965,1,1,0,...,[0],[I],3,1,3,3,True,False,False,3
996,2,497,I love this apple,"..)"")",0.998821,0.39173,0.608271,1,0,1,...,[],[],1,1,4,4,False,False,False,4
997,2,498,I love this apple,I like apples I like apples,0.998821,0.49518,0.504822,1,0,1,...,[0],[I],3,1,5,3,True,False,False,5
998,2,499,I love this apple,I likeApple I,0.998821,0.51566,0.515658,1,1,0,...,[0],[I],1,1,2,3,True,False,False,3


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)